## lightGBM version

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging
from sklearn.utils import shuffle

data_path = '../../../data'

answer_info = pd.read_csv(os.path.join(data_path, 'answer_info_0926.txt'), header=None, sep='\t')
#train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')

#train1.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
answer_info.columns = ['回答id', '问题id', '用户id', '回答创建时间', '回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数', '点赞数', '取赞数', '评论数', '收藏数', '感谢数', '举报数', '没有帮助数', '反对数']
print(answer_info.head(5))
answer_info['回答创建时间-day'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)

inv_info = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')
inv_info.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
inv_info['邀请创建时间-day'] = inv_info['邀请创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)

In [ ]:
result_path = '../3results'

In [ ]:
a = set([1,2,3,4])
b= set([2,3,4,5])
print(a.difference(b))

In [ ]:
def get_new_quest1(train_label, answer_info, inv_info, end_day):
    #question_in_answer = set(answer_info[answer_info['回答创建时间-day']<=end_day]['问题id'])
    question_in_inv = set(inv_info[inv_info['邀请创建时间-day']<=end_day]['问题id'])
    #old_question = list(question_in_answer.union(question_in_inv))
    old_question = question_in_inv
    
    question_in_label = set(train_label['qid'])
    question_all = list(question_in_label.union(old_question))
    new_question = list(question_in_label.difference(old_question))
    
    print(len(old_question), len(new_question))

    return old_question, new_question

def get_new_quest(train_label,answer_info, inv_info, end_day):
    #question_in_answer = set(answer_info[answer_info['回答创建时间-day']<=end_day]['问题id'])
    question_in_inv = set(inv_info[inv_info['邀请创建时间-day']<=end_day]['问题id'])
    #old_question = list(question_in_answer.union(question_in_inv))
    old_question = question_in_inv
    
    all_question = set(train_label['qid'])
    new_question = list(all_question.difference(old_question))
    
    print(len(old_question), len(new_question))

    return old_question, new_question


def get_new_users1(train_label, answer_info, inv_info, end_day):
    users_in_answer = set(answer_info[answer_info['回答创建时间-day']<=end_day]['用户id'])
    users_in_inv = set(inv_info[inv_info['邀请创建时间-day']<=end_day]['用户id'])
    old_users = set(users_in_answer.union(users_in_inv))
    
    users_in_label = set(train_label['uid'])
    users_all = set(users_in_label.union(old_users))
    new_users = set(users_in_label.difference(old_users))
    
    print(len(old_users), len(new_users))
    
    return old_users, new_users

def get_new_users(train_label, answer_info, inv_info, end_day):
     #question_in_answer = set(answer_info[answer_info['回答创建时间-day']<=end_day]['问题id'])
    user_in_inv = set(inv_info[inv_info['邀请创建时间-day']<=end_day]['用户id'])
    #old_question = list(question_in_answer.union(question_in_inv))
    old_user = user_in_inv
    
    all_user = set(train_label['uid'])
    new_user = list(all_user.difference(old_user))
    
    print(len(old_user), len(new_user))

    return old_user, new_user

def train_for4(train_all, test, train_name, sub_test, feature_cols):
    print("****************start"+train_name+"****************")
    
    if train_name == 'ouoq':
        print(feature_cols)
        print(len(feature_cols))
        
    elif train_name == 'nuoq':
        feature_cols = [x for x in feature_cols if x not in ('u_a_last_daytime', 'u_inv_count', 'u_inv_mean', 'u_inv_std', 'u_inv_sum', 'u_t_most_countword_id',
                                                            'u_a_i_diffday_max', 'u_a_i_diffday_mean', 'u_a_i_diffday_min', 'u_a_i_diffday_sum', 'u_a_i_diffhour_max', 'u_a_i_diffhour_mean', 
                                                             'u_a_i_diffhour_min', 'u_a_i_diffhour_sum', 'u_diff_qa_days_max', 'u_diff_qa_days_max_last14day', 'u_diff_qa_days_max_last3day', 
                                                             'u_diff_qa_days_max_last7day', 'u_diff_qa_days_mean', 'u_diff_qa_days_mean_last14day', 'u_diff_qa_days_mean_last3day', 
                                                             'u_diff_qa_days_mean_last7day', 'u_diff_qa_days_sum', 'u_diff_qa_days_sum_last14day', 'u_diff_qa_days_sum_last3day', 'u_diff_qa_days_sum_last7day',
                                                             'u_a_most_hour', 'u_a_most_hour_last1week', 'u_a_most_hour_last2week', 'u_a_most_hour_last3day', 'u_a_most_wkday',
                                                             'u_a_num', 'u_a_num_last14', 'u_a_num_last3', 'u_a_num_last7' ) ]
        print(feature_cols)
        
    elif train_name == 'nunq':
        feature_cols = [x for x in feature_cols if x not in ('u_a_last_daytime', 'u_inv_count', 'u_inv_mean', 'u_inv_std', 'u_inv_sum', 'u_t_most_countword_id',
                                                            'u_a_i_diffday_max', 'u_a_i_diffday_mean', 'u_a_i_diffday_min', 'u_a_i_diffday_sum', 'u_a_i_diffhour_max', 'u_a_i_diffhour_mean', 
                                                             'u_a_i_diffhour_min', 'u_a_i_diffhour_sum', 'u_diff_qa_days_max', 'u_diff_qa_days_max_last14day', 'u_diff_qa_days_max_last3day', 
                                                             'u_diff_qa_days_max_last7day', 'u_diff_qa_days_mean', 'u_diff_qa_days_mean_last14day', 'u_diff_qa_days_mean_last3day', 
                                                             'u_diff_qa_days_mean_last7day', 'u_diff_qa_days_sum', 'u_diff_qa_days_sum_last14day', 'u_diff_qa_days_sum_last3day', 'u_diff_qa_days_sum_last7day',
                                                             'u_a_most_hour', 'u_a_most_hour_last1week', 'u_a_most_hour_last2week', 'u_a_most_hour_last3day', 'u_a_most_wkday',
                                                             'u_a_num', 'u_a_num_last14', 'u_a_num_last3', 'u_a_num_last7' ) ]
        
        feature_cols = [x for x in feature_cols if x not in ('q_inv_count', 'q_inv_mean', 'q_inv_std', 'q_inv_sum', 
                                                            'q_ans_count', 'q_diff_qa_days_max', 'q_diff_qa_days_mean', 'q_diff_qa_days_sum', 'q_has_img_max', 'q_has_img_mean', 
                                                             'q_has_img_sum', 'q_has_video_max', 'q_has_video_mean', 'q_has_video_sum', 'q_is_dest_max', 'q_is_dest_mean', 'q_is_dest_sum', 
                                                             'q_is_good_max', 'q_is_good_mean', 'q_is_good_sum', 'q_is_rec_max', 'q_is_rec_mean', 'q_is_rec_sum', 
                                                             'q_reci_cheer_max', 'q_reci_cheer_mean', 'q_reci_cheer_sum', 'q_reci_comment_max', 'q_reci_comment_mean', 'q_reci_comment_sum', 
                                                             'q_reci_dis_max', 'q_reci_dis_mean', 'q_reci_dis_sum', 'q_reci_mark_max', 'q_reci_mark_mean', 'q_reci_mark_sum', 
                                                             'q_reci_no_help_max', 'q_reci_no_help_mean', 'q_reci_no_help_sum', 'q_reci_tks_max', 'q_reci_tks_mean', 'q_reci_tks_sum', 
                                                             'q_reci_uncheer_max', 'q_reci_uncheer_mean', 'q_reci_uncheer_sum', 
                                                             'q_reci_xxx_max', 'q_reci_xxx_mean', 'q_reci_xxx_sum', 'q_word_count_max', 'q_word_count_mean', 'q_word_count_sum')]    
        print(feature_cols)
        
    elif train_name == 'ounq':
        feature_cols = [x for x in feature_cols if x not in ('q_inv_count', 'q_inv_mean', 'q_inv_std', 'q_inv_sum', 
                                                            'q_ans_count', 'q_diff_qa_days_max', 'q_diff_qa_days_mean', 'q_diff_qa_days_sum', 'q_has_img_max', 'q_has_img_mean', 
                                                             'q_has_img_sum', 'q_has_video_max', 'q_has_video_mean', 'q_has_video_sum', 'q_is_dest_max', 'q_is_dest_mean', 'q_is_dest_sum', 
                                                             'q_is_good_max', 'q_is_good_mean', 'q_is_good_sum', 'q_is_rec_max', 'q_is_rec_mean', 'q_is_rec_sum', 
                                                             'q_reci_cheer_max', 'q_reci_cheer_mean', 'q_reci_cheer_sum', 'q_reci_comment_max', 'q_reci_comment_mean', 'q_reci_comment_sum', 
                                                             'q_reci_dis_max', 'q_reci_dis_mean', 'q_reci_dis_sum', 'q_reci_mark_max', 'q_reci_mark_mean', 'q_reci_mark_sum', 
                                                             'q_reci_no_help_max', 'q_reci_no_help_mean', 'q_reci_no_help_sum', 'q_reci_tks_max', 'q_reci_tks_mean', 'q_reci_tks_sum', 
                                                             'q_reci_uncheer_max', 'q_reci_uncheer_mean', 'q_reci_uncheer_sum', 
                                                             'q_reci_xxx_max', 'q_reci_xxx_mean', 'q_reci_xxx_sum', 'q_word_count_max', 'q_word_count_mean', 'q_word_count_sum')]        
        print(feature_cols)    
        
    else:
        return 0
    
    train_all = shuffle(train_all)
    X_train_all = train_all[feature_cols]
    y_train_all = train_all['label']

    fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=80)
    #scores=[]
    
    for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
        break
    X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
    model_lgb = LGBMClassifier(n_estimators=2000, n_jobs=-1, objective='binary', seed=1500, silent=True)  #scale_pos_weight=2.5 when data=data_all
    
    model_lgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=300)
    
    del X_train_all
    print(pd.DataFrame({
            'column': feature_cols,
            'importance': model_lgb.feature_importances_
                    }).sort_values(by='importance', ascending=False))

    
    model_lgb.booster_.save_model('model'+train_name+'.txt')
    #assert sub_size == test_res.shape[0]
    sub_test['label'] = model_lgb.predict_proba(test[feature_cols])[:, 1]
    sub_test.to_hdf('model_saved/sub_test'+train_name+'.hd5', key='data')
    print("**********************end"+train_name+"*******************************")

    return sub_test ###['uid','qid','time','label','index']



In [ ]:
# 减少内存占用
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            
            if str(col_type)[:5] == 'float':
                df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
import pickle
feature_path = '../1pre_features'
with open(os.path.join(feature_path, 'que_topic_all_count_train.pkl'), 'rb') as file:
    train_q_t_c = pickle.load(file)
    train_q_t_c = reduce_mem_usage(train_q_t_c)
print(train_q_t_c.info())

In [ ]:
print(train_q_t_c.head())

In [ ]:
print(list(train_q_t_c.columns.values))

In [ ]:
with open(os.path.join(feature_path, 'user_topic_all_count_train.pkl'), 'rb') as file:
    train_u_t_c = pickle.load(file)[['uid', 'u_t_a_count_sum', 'u_t_a_count_mean','u_t_a_num']]#, 'u_t_a_count_max', 'u_t_a_count_std', 'u_t_a_count_min']]
    train_u_t_c = reduce_mem_usage(train_u_t_c)
print(train_u_t_c.info())


In [ ]:
import pickle

#train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')
test1 = pd.read_csv(os.path.join(data_path, 'invite_info_evaluate_1_0926.txt'), header=None, sep='\t')
test1.columns = ['qid', 'uid', 'time']

feature_path = '../1pre_features'
data = pd.read_hdf(os.path.join(feature_path,'data_dropout.h5'),key='data').reset_index()
print(data.info())
train_len = 1504120
#train_len = 2593669
#train_len = 1504441 #dropout=1
#train_len = 1504630 #dropout=2

'''修改'''
# # add: user_topic_answer_count
# with open(os.path.join(feature_path, 'u_topic_a_zishu_train.pkl'), 'rb') as file:
#     train_u_t_c = pickle.load(file)
# #print(question_topic.head(5))
# with open(os.path.join(feature_path, 'u_topic_a_zishu_val.pkl'), 'rb') as file:
#     test_u_t_c = pickle.load(file)
# train_utc = pd.merge(data[:train_len], train_u_t_c, on='uid', how='left')
# test_utc = pd.merge(data[train_len:], test_u_t_c, on='uid', how='left')
# data = pd.concat([train_utc, test_utc], axis=0, sort=True)  
# del train_utc, test_utc

# add: user_topic_answer_count
with open(os.path.join(feature_path, 'user_topic_all_count_train.pkl'), 'rb') as file:
    train_u_t_c = pickle.load(file)[['uid', 'u_t_a_count_sum', 'u_t_a_count_mean','u_t_a_num', 'u_t_a_count_max', 'u_t_a_count_std', 'u_t_a_count_min']]
    train_u_t_c = reduce_mem_usage(train_u_t_c)
#print(question_topic.head(5))
with open(os.path.join(feature_path, 'user_topic_all_count_test.pkl'), 'rb') as file:
    test_u_t_c = pickle.load(file)[['uid', 'u_t_a_count_sum', 'u_t_a_count_mean','u_t_a_num', 'u_t_a_count_max', 'u_t_a_count_std', 'u_t_a_count_min']]
    test_u_t_c = reduce_mem_usage(test_u_t_c)
train_utc = pd.merge(data[:train_len], train_u_t_c, on='uid', how='left')
test_utc = pd.merge(data[train_len:], test_u_t_c, on='uid', how='left')
data = pd.concat([train_utc, test_utc], axis=0, sort=True)  
data['u_t_a_q_prt'] = data['u_t_a_num'] / data['u_ans_count']
del train_utc, test_utc, train_u_t_c, test_u_t_c
print("user topic loaded")

# add: question_topic_answer_count
with open(os.path.join(feature_path, 'que_topic_all_count_train.pkl'), 'rb') as file:
    train_q_t_c = pickle.load(file)#'q_t_a_inv_mean_mean', 'q_t_a_inv_mean_max', 'q_t_a_inv_mean_min', 'q_t_a_inv_mean_std',]]#['uid', 'u_t_a_count_sum', 'u_t_a_count_mean', 'u_t_a_count_max', 'u_t_a_count_std', 'u_t_a_count_min']
    #train_q_t_c = reduce_mem_usage(train_q_t_c)
#print(question_topic.head(5))
with open(os.path.join(feature_path, 'que_topic_all_count_val.pkl'), 'rb') as file:
    test_q_t_c = pickle.load(file)#'q_t_a_inv_mean_mean', 'q_t_a_inv_mean_max', 'q_t_a_inv_mean_min', 'q_t_a_inv_mean_std',]]#['uid', 'u_t_a_count_sum', 'u_t_a_count_mean', 'u_t_a_count_max', 'u_t_a_count_std', 'u_t_a_count_min']
    #test_q_t_c = reduce_mem_usage(test_q_t_c)
train_qtc = pd.merge(data[:train_len], train_q_t_c, left_on='qid', right_on='问题id', how='left').drop(['问题id'], axis=1, inplace=False)
test_qtc = pd.merge(data[train_len:], test_q_t_c, left_on='qid', right_on='问题id', how='left').drop(['问题id'], axis=1, inplace=False)
data = pd.concat([train_qtc, test_qtc], axis=0, sort=True)  
del train_qtc, test_qtc, train_q_t_c, test_q_t_c
print("question topic loaded")


# add: question_basic_extra_feats
question_info1 = pd.read_hdf(os.path.join(feature_path,'question_info1.hd5'), key='data')
data = pd.merge(data, question_info1, left_on='qid', right_on='问题id', how='left').drop('问题id', axis=1)
data['diff_qi_days'] = data['day'] - data['q_day']
data['diff_qi_hours'] = data['hour'] - data['q_hour']
data.loc[:,'diff_qi_hours'] += 23
drop_feat = ['q_day','q_hour']
data  = data.drop(drop_feat, axis=1)
print(data.info())

# add: user_question_topic_embedding_similarity_feats
with open(os.path.join(feature_path,'test_cosine_similarity_without_5_10_drop_embedding.pkl'), 'rb') as file:
    test_cosine_sim = pickle.load(file).drop_duplicates(subset=['uid','qid'], keep='first', inplace=False)
with open(os.path.join(feature_path,'train_cosine_similarity_without_5_10_drop_embedding.pkl'), 'rb') as file:
    train_cosine_sim = pickle.load(file).drop_duplicates(subset=['uid','qid'], keep='first', inplace=False)
train_sim = pd.merge(data[:train_len], train_cosine_sim, on=['uid','qid'], how='left')
test_sim = pd.merge(data[train_len:], test_cosine_sim, on=['uid','qid'], how='left')
data = pd.concat([train_sim, test_sim], axis=0, sort=True)  
del train_sim, test_sim

# #add: xgb_score
# xgb_results = pd.read_csv('result_xgb.txt', sep='\t')
# xgb_results.columns = ['qid','uid','i_time','xgb_score']
# xgb_results['day'] = xgb_results['i_time'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
# xgb_results['hour'] = xgb_results['i_time'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
# del xgb_results['i_time']
# xgb_results = xgb_results.drop_duplicates(subset=['uid','qid','day','hour'], keep='first', inplace=False)
# data = pd.merge(data, xgb_results, on=['uid','qid','day','hour'], how='left')

# #add: xgb_score
# lgb1_results = pd.read_csv('resultlgbm.txt', sep='\t')
# lgb1_results.columns = ['uid','qid','i_time','lgb1_score']
# lgb1_results['day'] = lgb1_results['i_time'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
# lgb1_results['hour'] = lgb1_results['i_time'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
# del lgb1_results['i_time']
# lgb1_results = lgb1_results.drop_duplicates(subset=['uid','qid','day','hour'], keep='first', inplace=False)
# data = pd.merge(data, lgb1_results, on=['uid','qid','day','hour'], how='left')

# #add: dnn_score_output

# dnn_results_train = pd.read_csv('submit_train_all1.txt', sep='\t')
# dnn_results_train.columns = ['qid','uid','i_time','dnn_score']
# dnn_results_test = pd.read_csv('submit_test_all2.txt', sep='\t')
# dnn_results_test.columns = ['qid','uid','i_time','dnn_score']
# dnn_results = pd.concat([dnn_results_train, dnn_results_test], axis=0, sort=True)

# dnn_results['day'] = dnn_results['i_time'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
# dnn_results['hour'] = dnn_results['i_time'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
# del dnn_results['i_time']
# dnn_results = dnn_results.drop_duplicates(subset=['uid','qid','day','hour'], keep='first', inplace=False)
# data = pd.merge(data, dnn_results, on=['uid','qid','day','hour'], how='left')

# add: user_answer_side_feats
user_answer_feats_val = pd.read_hdf(os.path.join(feature_path,'user_answer_val.h5'),key='data')
user_answer_feats_train = pd.read_hdf(os.path.join(feature_path,'user_answer_train.h5'),key='data')
# user_answer_feats_train = user_answer_feats_train[['用户id','u_a_i_diffhour_mean','u_a_i_diffhour_sum',
#                                                    'u_a_i_diffhour_max','u_a_i_diffhour_min','u_a_i_diffday_mean',
#                                                    'u_a_i_diffday_sum','u_a_i_diffday_max','u_a_i_diffday_min',
#                                                    '用户习惯回答时间-hour','用户最近回答数-3天','用户最近回答数-7天','用户最近回答数-14天',
#                                                    '用户习惯回答时间-wkday']]#,'用户上次回答时间-day']]#,
# user_answer_feats_val = user_answer_feats_val[['用户id','u_a_i_diffhour_mean','u_a_i_diffhour_sum',
#                                                    'u_a_i_diffhour_max','u_a_i_diffhour_min','u_a_i_diffday_mean',
#                                                    'u_a_i_diffday_sum','u_a_i_diffday_max','u_a_i_diffday_min',
#                                                    '用户习惯回答时间-hour','用户最近回答数-3天','用户最近回答数-7天','用户最近回答数-14天',
#                                               '用户习惯回答时间-wkday']]#,'用户上次回答时间-day']]#' ,

# user_answer_feats_train = user_answer_feats_train.rename(columns={'用户习惯回答时间-hour':'u_most_a_hour','用户最近回答数-3天':'u_a_num_latest3',
#                                         '用户最近回答数-7天':'u_a_num_latest7','用户最近回答数-14天':'u_a_num_latest14',
#                                         '用户习惯回答时间-wkday':'u_most_a_wkday','用户上次回答时间-day':'u_last_a_time'})#,
# user_answer_feats_val = user_answer_feats_val.rename(columns={'用户习惯回答时间-hour':'u_most_a_hour','用户最近回答数-3天':'u_a_num_latest3',
#                                         '用户最近回答数-7天':'u_a_num_latest7','用户最近回答数-14天':'u_a_num_latest14',
#                                         '用户习惯回答时间-wkday':'u_most_a_wkday','用户上次回答时间-day':'u_last_a_time'})#

#print(user_answer_feats_train[['用户id','u_last_a_time']])

train2 = pd.merge(data[:train_len], user_answer_feats_train, left_on='uid', right_on='用户id', how='left' ).drop('用户id',axis=1)
test2 = pd.merge(data[train_len:],user_answer_feats_val, left_on='uid', right_on='用户id', how='left' ).drop('用户id',axis=1)
train2['u_a_last_daytime'] = train2['day'] - train2['u_a_last_daytime']
test2['u_a_last_daytime'] = test2['day'] - test2['u_a_last_daytime']

data = pd.concat([train2, test2], axis=0, sort=True)
del train2, test2

# add: user_topic_side_feats
user_topic_feats = pd.read_hdf(os.path.join(feature_path,'user_topic_feat.h5'),key='data')
data = pd.merge(data,user_topic_feats, left_on='uid', right_on='author_id',how='left').drop('author_id',axis=1)
print("loaded topic feats")

user_question_topic = pd.read_hdf(os.path.join(feature_path,'member_question_feat.h5'), key='data')
user_question_topic = user_question_topic.rename(columns={'用户问题的话题关注重叠数':'u_q_topic_att_num','用户问题的话题感兴趣重叠数':'u_q_topic_ints_num'})

data = pd.merge(data, user_question_topic, left_on=['uid','qid'], right_on=['author_id','question_id'], how='left').drop(['author_id','question_id'],axis=1)
print("loaded user_question topics")

# add: question_side_feats
# user_qa_days_feats_train = pd.read_hdf(os.path.join(feature_path,'../u_qa_day_feats_train.h5'),key='data')
# user_qa_days_feats_val = pd.read_hdf(os.path.join(feature_path,'../u_qa_day_feats_val.h5'),key='data')
# print("loaded u_qa_days")

'''修改end'''

In [ ]:
print(data.columns.values)
print(len(data.columns.values))

In [ ]:
def basic_count1(data, count_fea):
    for feat in count_fea: 
        data[feat] = data[feat].fillna(0)
        data[feat] = (data[feat] - data[feat].min()) / (np.percentile(np.array(data[feat]), 90) - data[feat].min())
    return data

data1 = basic_count1(data, ['u_inv_count','q_inv_count',
                            'q_ans_count', 'u_q_topic_att_num', 'u_q_topic_ints_num', 
                           'u_ans_count','q_title_num', 'u_a_num_last14','u_a_num','u_a_num_last3','num_atten_topic', 'num_interest_topic',
                           'u_q_topic_att_num', 'u_q_topic_ints_num'])

In [ ]:
def basic_count2(data, count_fea):
    for feat in count_fea: 
        data[feat] = data[feat].fillna(0)
        data[feat] = np.log10(data[feat])
    return data

data1 = basic_count1(data1, ['q_inv_std','q_t_a_count_std','q_t_a_count_std',
                           'q_t_a_inv_mean_std', 'q_t_a_mark_count_std','q_t_a_neg_inv_mean_std',
                           'q_t_a_thank_count_std','u_inv_std','std_interest_values','min_interest_values', 'max_interest_values', 'mean_interest_values',
                            'q_t_a_inv_count_mean', 'q_t_a_inv_mean_max', 'q_t_a_inv_mean_mean', 'q_t_a_inv_mean_min','q_t_a_inv_std_mean', 'q_t_a_inv_sum_mean',
                            'u_inv_mean', 
                           ])

In [ ]:
feature_cols = [x for x in data.columns if x not in ('u_ans_count','dnn_score','u_a_last_daytime','xgb_score', 'lgb1_score','index','uid', 'qid','dropoutrate', 'label', 'dt', 'day', )]#'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count')]
#feature_cols = [x for x in feature_cols if x not in ('u_a_mean_hour_last1week', 'u_a_mean_hour_last2week', 'u_a_mean_hour_last3day', 'u_a_most_hour', 'u_a_most_hour_last1week', 'u_a_most_hour_last2week', 'u_a_most_hour_last3day', 'u_a_most_wkday', 'u_a_num', 'u_a_num_last14', 'u_a_num_last3',)]#'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count')]

In [ ]:
print(len(feature_cols))

In [ ]:
220

#### ------------------------------**分支1：整体训练**---------------------------

In [ ]:
test_all = data.iloc[train_len:]
assert len(test_all) == test1.shape[0]
train_all = data.iloc[:train_len]
logging.info("train shape %d, test shape %s", train_len, test_all.shape)

sub = test1.copy().reset_index()

result = train_for4(train_all, test_all, 'ouoq', sub, feature_cols)
result = result.drop(['index'], axis=1, inplace=False)

In [ ]:
print(result.head())
result.to_csv(os.path.join(result_path, 'result_withouttime222.txt'), index=None, header=None, sep='\t')
print("result written")

In [ ]:
# 绘制重要性因子的图表
import lightgbm as lgb
import matplotlib.pylab as plt

lgbm111 = lgb.Booster(model_file='model_saved/model.txt')
plt.figure(figsize=(2400,1200))
lgb.plot_importance(lgbm111, max_num_features=30)
plt.title("Featurertances")
plt.show()

#### ------------------------------**分支1：整体训练结束**-----------------------------------

#### -----------------------**分支2：分类训练**---------------------------

In [ ]:
print(data1.head(3))

test_all = data1.iloc[train_len:]
print(len(test_all))
assert len(test_all) == test1.shape[0]
train_all = data1.iloc[:train_len]
logging.info("train shape %d, test shape %s", train_len, test_all.shape)

train_old_users, train_new_users = get_new_users(train_all, answer_info, inv_info, 3860)
test_old_users, test_new_users = get_new_users(test_all, answer_info, inv_info, 3868)
train_old_question, train_new_question = get_new_quest(train_all, answer_info, inv_info, 3860)
test_old_question, test_new_question = get_new_quest(test_all, answer_info, inv_info, 3868)

train_ouoq = train_all[train_all['uid'].isin(train_old_users)][train_all['qid'].isin(train_old_question)]
train_nuoq = train_all[train_all['uid'].isin(train_new_users)][train_all['qid'].isin(train_old_question)]
train_ounq = train_all[train_all['uid'].isin(train_old_users)][train_all['qid'].isin(train_new_question)]
train_nunq = train_all[train_all['uid'].isin(train_new_users)][train_all['qid'].isin(train_new_question)]

# train_oq = train_all[train_all['qid'].isin(train_old_question)]
# train_nq = train_all[train_all['qid'].isin(train_new_question)]

test_ouoq = test_all[test_all['uid'].isin(test_old_users)][test_all['qid'].isin(test_old_question)]
test_nuoq = test_all[test_all['uid'].isin(test_new_users)][test_all['qid'].isin(test_old_question)]
test_nunq = test_all[test_all['uid'].isin(test_new_users)][test_all['qid'].isin(test_new_question)]
test_ounq = test_all[test_all['uid'].isin(test_old_users)][test_all['qid'].isin(test_new_question)]

# test_oq = test_all[test_all['qid'].isin(test_old_question)]
# test_nq = test_all[test_all['qid'].isin(test_new_question)]

sub = test1.copy().reset_index()
#sub_size = len(sub)
sub['index1'] = sub.index
print(sub.info())

sub_ouoq = sub[sub['uid'].isin(test_old_users)][sub['qid'].isin(test_old_question)]
sub_ounq = sub[sub['uid'].isin(test_old_users)][sub['qid'].isin(test_new_question)]
sub_nuoq = sub[sub['uid'].isin(test_new_users)][sub['qid'].isin(test_old_question)]
sub_nunq = sub[sub['uid'].isin(test_new_users)][sub['qid'].isin(test_new_question)]

# sub_oq = sub[sub['qid'].isin(test_old_question)]
# sub_nq = sub[sub['qid'].isin(test_new_question)]

# sub.to_csv('result.txt', index=None, header=None, sep='\t')

pd.set_option('display.max_rows', None)

In [ ]:
print(len(test_old_question))

In [ ]:
print(train_all.columns.values)

In [ ]:
feature_cols1 = [x for x in data.columns if x not in ('u_a_last_daytime', 'xgb_score', 'lgb1_score','index','uid', 'qid','dropoutrate', 'label', 'dt', 'day', )]#'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count')]

In [ ]:
sub_oq = train_for4(train_all, test_oq, 'ouoq', sub_oq, feature_cols1)

In [ ]:
sub_nq = train_for4(train_all, test_nq, 'ounq', sub_nq, feature_cols1)

In [ ]:
sub_result = pd.concat([sub_oq, sub_nq], axis=0)  
sub_result.sort_values(by="index1" , ascending=True, inplace=True)
print(sub_result.head(15))
sub_result2 = sub_result.drop(['index','index1'], axis=1)
print(sub_result2.head(5))

sub_result2.to_csv('result.txt', index=None, header=None, sep='\t')

In [ ]:
#print(train_ouoq[train_ouoq['xgb_score']!=None]['xgb_score'].head(30))

### ---------------------分隔线----------------------

In [ ]:
sub_ouoq = train_for4(train_all, test_ouoq, 'ouoq', sub_ouoq, feature_cols)

In [ ]:
sub_ounq = train_for4(train_all, test_ounq, 'ounq', sub_ounq, feature_cols)

In [ ]:
sub_nuoq = train_for4(train_all, test_nuoq, 'nuoq', sub_nuoq, feature_cols)

In [ ]:
sub_nunq = train_for4(train_all, test_nunq, 'nunq', sub_nunq, feature_cols)

In [ ]:
sub_result = pd.concat([sub_ouoq, sub_nuoq], axis=0)  
sub_result = pd.concat([sub_result, sub_ounq], axis=0)  
sub_result = pd.concat([sub_result, sub_nunq], axis=0)  
sub_result.sort_values(by="index1" , ascending=True, inplace=True)
print(sub_result.head(15))
sub_result2 = sub_result.drop(['index','index1'], axis=1)
print(sub_result2.head(5))

sub_result2.to_csv('result.txt', index=None, header=None, sep='\t')

#### --------------------------------------**分支2：分类训练结束**-------------------------------------